<a href="https://colab.research.google.com/github/ChnssA/deep-learning-subject/blob/main/aml25010Lab_Sheet_%E2%80%93_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

class MPNeuron:
    def __init__(self, threshold):
        self.threshold = threshold

    def predict(self, inputs):
        # Calculate the sum of inputs
        total_sum = np.sum(inputs)
        # Apply threshold logic
        return 1 if total_sum >= self.threshold else 0

# Define the input combinations for 2-bit logic (00, 01, 10, 11)
inputs = [[0, 0], [0, 1], [1, 0], [1, 1]]

# 1. AND Gate (Threshold = 2)
and_neuron = MPNeuron(threshold=2)
print("AND Gate Output:")
for i in inputs:
    print(f"Input: {i} -> Output: {and_neuron.predict(i)}")

print("-" * 20)

# 2. OR Gate (Threshold = 1)
or_neuron = MPNeuron(threshold=1)
print("OR Gate Output:")
for i in inputs:
    print(f"Input: {i} -> Output: {or_neuron.predict(i)}")

print("-" * 20)

# 3. NOR Gate Logic
# For NOR, we output 1 only if the sum is 0.
# We can model this by saying: if sum >= 1, output 0 (Inverse of OR).
print("NOR Gate Output:")
for i in inputs:
    # A simple way to represent NOR with MP logic:
    res = 1 if np.sum(i) == 0 else 0
    print(f"Input: {i} -> Output: {res}")

AND Gate Output:
Input: [0, 0] -> Output: 0
Input: [0, 1] -> Output: 0
Input: [1, 0] -> Output: 0
Input: [1, 1] -> Output: 1
--------------------
OR Gate Output:
Input: [0, 0] -> Output: 0
Input: [0, 1] -> Output: 1
Input: [1, 0] -> Output: 1
Input: [1, 1] -> Output: 1
--------------------
NOR Gate Output:
Input: [0, 0] -> Output: 1
Input: [0, 1] -> Output: 0
Input: [1, 0] -> Output: 0
Input: [1, 1] -> Output: 0


In [4]:
import numpy as np
import pandas as pd
import sklearn.datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression

# 1. Load the Dataset
breast_cancer = sklearn.datasets.load_breast_cancer()
X = breast_cancer.data
Y = breast_cancer.target
features = breast_cancer.feature_names

# Convert to DataFrame
data = pd.DataFrame(breast_cancer.data, columns=breast_cancer.feature_names)
data['class'] = breast_cancer.target

# 2. MP Neuron Class Definition
class MPNeuron:
    def __init__(self):
        self.b = None

    def model(self, x):
        return (sum(x) >= self.b)

    def predict(self, X):
        Y = []
        for x in X:
            result = self.model(x)
            Y.append(result)
        return np.array(Y)

    def fit(self, X, Y):
        accuracy = {}
        for b in range(X.shape[1] + 1):
            self.b = b
            Y_pred = self.predict(X)
            accuracy[b] = accuracy_score(Y_pred, Y)

        best_b = max(accuracy, key=accuracy.get)
        self.b = best_b
        return best_b, accuracy[best_b]

# 3. Data Preprocessing & Binarization Function
def prepare_and_test(X_train, X_test, Y_train, Y_test, label):
    # MP Neuron requires binary input. We binarize using the mean of each feature.
    X_bin_train = X_train.apply(pd.cut, bins=2, labels=[1, 0]).values # 1 for 'healthy/low' 0 for 'risk' or vice versa
    X_bin_test = X_test.apply(pd.cut, bins=2, labels=[1, 0]).values

    mp_neuron = MPNeuron()
    best_threshold, train_acc = mp_neuron.fit(X_bin_train, Y_train)

    # Test Evaluation
    Y_test_pred = mp_neuron.predict(X_bin_test)
    test_acc = accuracy_score(Y_test_pred, Y_test)

    print(f"--- {label} ---")
    print(f"Optimal Threshold: {best_threshold}")
    print(f"Test Accuracy: {test_acc:.4f}\n")
    return test_acc

# Split data
X_train, X_test, Y_train, Y_test = train_test_split(data.drop('class', axis=1), data['class'], test_size=0.1, stratify=data['class'], random_state=1)

# --- a & b: Analyze Unscaled vs Scaled MP Neuron ---
# Unscaled
acc_unscaled = prepare_and_test(X_train, X_test, Y_train, Y_test, "Unscaled MP Neuron")

# Min-Max Scaling [cite: 11]
scaler_mm = MinMaxScaler()
X_train_mm = pd.DataFrame(scaler_mm.fit_transform(X_train), columns=features)
X_test_mm = pd.DataFrame(scaler_mm.transform(X_test), columns=features)
acc_mm = prepare_and_test(X_train_mm, X_test_mm, Y_train, Y_test, "Min-Max Scaled MP Neuron")

# Standardization [cite: 11]
scaler_std = StandardScaler()
X_train_std = pd.DataFrame(scaler_std.fit_transform(X_train), columns=features)
X_test_std = pd.DataFrame(scaler_std.transform(X_test), columns=features)
acc_std = prepare_and_test(X_train_std, X_test_std, Y_train, Y_test, "Standardized MP Neuron")

# --- c: Compare with Logistic Regression ---
lr_model = LogisticRegression(max_iter=10000)
lr_model.fit(X_train_std, Y_train)
lr_acc = accuracy_score(lr_model.predict(X_test_std), Y_test)

print(f"--- Comparison ---")
print(f"Best MP Neuron Accuracy: {max(acc_unscaled, acc_mm, acc_std):.4f}")
print(f"Logistic Regression Accuracy: {lr_acc:.4f}")

--- Unscaled MP Neuron ---
Optimal Threshold: 28
Test Accuracy: 0.7895

--- Min-Max Scaled MP Neuron ---
Optimal Threshold: 28
Test Accuracy: 0.7895

--- Standardized MP Neuron ---
Optimal Threshold: 28
Test Accuracy: 0.7895

--- Comparison ---
Best MP Neuron Accuracy: 0.7895
Logistic Regression Accuracy: 0.9825
